In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import bifacialvf

In [5]:
filestarter = "irr_1axis_2023-09-16_" # The string your results file names start with.
filelist = sorted(os.listdir(os.path.join(testfolder, 'results')))
prefixed = [filename for filename in filelist if filename.startswith(filestarter)]
# print(prefixed)
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Ap_1 = []
Ap_2 = []
Ap_3 = []
Ap_4 = []
Ap_5 = []

print('{} files in the directory'.format(filelist.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')

    try:
        Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[i])[0])
        resultsDF = br.load.read1Result(os.path.join(testfolder, 'results', prefixed[i]))
        Ap_1.append(resultsDF['Wm2Back'][0])
        Ap_2.append(resultsDF['Wm2Back'][1])
        Ap_3.append(resultsDF['Wm2Back'][2])
        Ap_4.append(resultsDF['Wm2Back'][3])
        Ap_5.append(resultsDF['Wm2Back'][4])

        filenamed.append(prefixed[i])
    except:
        print(" FAILED ", i, prefixed[i])
        faillist.append(prefixed[i])
        

resultsdf = pd.DataFrame(list(zip(Ap_1, Ap_2, Ap_3, Ap_4, Ap_5)), columns = ['Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5'])

resultsdf['Datetime'] = Datetime
resultsdf['Datetime'] = pd.to_datetime(resultsdf['Datetime'], format ='%Y-%m-%d_%H%M')

147 files in the directory
47 groundscan files in the directory


In [6]:
# Index the modeled data to the datetime column and add suffix before concatenating with the measured data. This will allow the "inner" join of the two dataframes using the indexed datetime and easy pivoting of the data afterwards.
modeled = resultsdf.set_index('Datetime')
modeled_suffix = modeled.add_suffix(".modeled")
modeled['datatype']="modeled"

# Load the measured data
measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1) # The field pyranometer data
srrl_weather = pd.read_csv(str(Path().resolve() / 'WeatherFiles' /  'PSM3_15T.csv'), header = 2) # Data from SRRL

# Create a Datetime variable and set it as the index
srrl_weather['date'] = srrl_weather[['Year','Month','Day']].astype(str).agg('-'.join, axis=1) # Couldn't think of a proper way to chain this process into a single command. If you think of something, please let me know
srrl_weather['time'] = srrl_weather[['Hour','Minute']].astype(str).agg(':'.join, axis=1)
srrl_weather['Datetime'] = pd.to_datetime(srrl_weather[['date','time']].agg(" ".join, axis=1))
srrl_weather.set_index('Datetime', inplace=True)

# Create a Datetime variable for the measured data using the existing 'TIMESTAMP' column.
measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
measured = measured.drop(["TIMESTAMP", "RECORD"], axis = 'columns') # drop this column so the resample command does not confuse TIMESTAMP and Datetime columns
measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='left', closed='right').mean() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.

# Add suffix to the columns in the measured dataframe before concatenating it with the modeled dataframe
measured_suffix = measured.add_suffix(".measured")
measured["datatype"]="measured"

modeledandmeasured_suffix = pd.concat([modeled_suffix, measured_suffix], axis=1, join='inner')
modeledandmeasured_melt = pd.wide_to_long(df = modeledandmeasured_suffix.reset_index(), stubnames= ["Ap_1", "Ap_2", "Ap_3", "Ap_4", "Ap_5","ST_con", "ST_pv_bp", "ST_pv_is", "PAR_con", "PAR_pv", "Batt_Volt", "PTemp_C"], i = "Datetime", j = "datatype", sep = ".", suffix = "\D+").reset_index().set_index('Datetime')

modeledandmeasured_melt2 = modeledandmeasured_melt[['datatype', 'Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5']].reset_index().melt(id_vars = ['Datetime', 'datatype'], var_name = 'position', value_name= 'value').reset_index()
modeledandmeasured_melt3 = modeledandmeasured_melt2.pivot(index=['Datetime', 'position'], columns = 'datatype', values = 'value')


# Sum up the irradiance values and divide it by 4 to get accumulated daily radiation
modeledandmeasured_sum = modeledandmeasured_melt2.reset_index().groupby(['position', 'datatype']).agg({"value":"sum"})['value'].transform(lambda x:x/4)

# Grab just the measured
ghipar = modeledandmeasured_melt.loc[modeledandmeasured_melt['datatype']=='measured'][['Ap_3', 'PAR_pv', 'PAR_con']]
srrlandghipar = pd.concat([ghipar, srrl_weather], axis=1, join="inner")[['Ap_3','GHI', 'PAR_pv', 'PAR_con']].rename(columns={'Ap_3':'ghi_pv','GHI':'ghi_con','PAR_pv':'par_pv','PAR_con':'par_con'}).reset_index()
srrlandghipar = pd.wide_to_long(df = srrlandghipar.reset_index(), stubnames=['ghi','par'], i = "Datetime", j = "location", sep = "_", suffix = "\D+").reset_index()
print(srrlandghipar)


              Datetime location  index         ghi         par
0  2023-09-16 06:00:00       pv      0   38.265983   91.092239
1  2023-09-16 06:15:00       pv      1   30.877190   99.422738
2  2023-09-16 06:30:00       pv      2   22.926993   82.363319
3  2023-09-16 06:45:00       pv      3   80.726105  210.091831
4  2023-09-16 07:00:00       pv      4   55.949033  153.040217
..                 ...      ...    ...         ...         ...
89 2023-09-16 16:30:00      con     42  303.666933  563.465340
90 2023-09-16 16:45:00      con     43  250.607267  455.401733
91 2023-09-16 17:00:00      con     44  198.799933  344.581373
92 2023-09-16 17:15:00      con     45  148.739067  162.476525
93 2023-09-16 17:30:00      con     46  100.886640   73.462101

[94 rows x 5 columns]


In [7]:
# # Tidy the data
# modeled_melt = resultsdf.melt(id_vars = ['Datetime'], var_name = 'position', value_name = 'value')
# modeled_melt['datatype'] = 'modeled'

# # Load the measured data
# measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1)
# # print(resultsdf_melt.info)
# # measured.info

# measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
# measured = measured.drop(['TIMESTAMP'], axis='columns').set_index('Datetime')
# measured_15Tmean = measured.resample('15T', axis = 'index', label='right', closed='right').mean()
# # measured_select = measured[['Datetime', 'Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5']]
# measured_melt = measured.reset_index()[['Datetime', 'Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5']].melt(
#     id_vars = ['Datetime'],
#     var_name = 'position', 
#     value_name = 'value')
# measured_melt['datatype'] = 'measured'

# # combined = pd.merge(resultsdf_melt, measured_melt, how = 'left', on = 'Datetime')
# measured_melt = pd.merge(modeled_melt[['Datetime']], measured_melt, how = 'left', on = 'Datetime')
# measured_melt.drop_duplicates(inplace=True)
# combined = pd.concat([modeled_melt, measured_melt])


# # Make wider for another plot
# # combined_wider = combined.pivot(index=['Datetime', 'position'], columns='datatype', values = 'value').reset_index().set_index('Datetime')
# combined_wider = combined.pivot(index=['Datetime', 'position'], columns='datatype', values = 'value').reset_index()
# # print(combined_wider)
# combined_wider['time'] = pd.to_datetime(combined_wider['Datetime'], format = '%H:%M')
# combined_wider = combined_wider.set_index("Datetime")
# # Sum up the irradiance values
# # combined_sum = combined.groupby(['position', 'datatype']).agg(np.sum())
# combined['totalrad'] = combined.groupby(['position', 'datatype'])['value'].transform(lambda x: x.sum() / 4)
# combined_summary = combined[['position','datatype','totalrad']].drop_duplicates()
# print(combined_summary)


In [8]:
# look at residuals (MBD, RMSE) based on Grear, Gpoa and Gtotal_modeled. From B. Marion Solar Energy 2016
def MBD(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*((1/m)*sum(df.model-df.meas))/df.meas.mean()
    return out

def RMSE(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*np.sqrt(1/m*sum((df.model-df.meas)**2))/df.meas.mean()
    return out

# residuals absolute output (not %) 
def MBD_abs(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = ((1/m)*sum(df.model-df.meas))
    return out

def RMSE_abs(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = np.sqrt(1/m*sum((df.model-df.meas)**2))
    return out

In [9]:
# Create the empty arrays (or lists? I don't know all the object types in python) to save the result of the looped error analysis
MBD_result = []
RMSE_result = []
MBD_abs_result = []
RMSE_abs_result = []
faillist = []

# Grab the list of column names
colnames = list(modeledandmeasured_suffix.columns.values)
modposition = []
measposition = []

# Setting the index to 0 prior to running the for-loop
i = 0

# For-looping the error analyses
for i in range (0, 5): # The index range was determined after looking at the 
    try:
        MBD_result.append(MBD(modeledandmeasured_suffix[colnames[i+5]], modeledandmeasured_suffix[colnames[i]])) # The modeled columns and the measured counterparts are five columns apart, hence "i" and "i+5"
        RMSE_result.append(RMSE(modeledandmeasured_suffix[colnames[i+5]], modeledandmeasured_suffix[colnames[i]]))
        MBD_abs_result.append(MBD_abs(modeledandmeasured_suffix[colnames[i+5]], modeledandmeasured_suffix[colnames[i]]))
        RMSE_abs_result.append(RMSE_abs(modeledandmeasured_suffix[colnames[i+5]], modeledandmeasured_suffix[colnames[i]]))

        modposition.append(colnames[i])
        measposition.append(colnames[i+5])
    except:
        print(" FAILED ", i, colnames[i])
        faillist.append(colnames[i])

# Join the arrays into a dataframe
validationdf = pd.DataFrame(list(zip(modposition, measposition, MBD_result, RMSE_result, MBD_abs_result, RMSE_abs_result)), columns = ['Modeled', 'Measured', 'MBD', 'RMSE', 'MBD_abs', 'RMSE_abs'])    

# Export the result into a csv
validationdf.to_csv('out6.csv',index=False)

In [63]:
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacialVF version ", bifacialvf.__version__)

Working on a  Windows 10
Python version  3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]
Pandas version  2.2.2
bifacialVF version  0.1.10.dev1+g107a777


In [3]:
# Variables
sazm = 180                  # PV Azimuth(deg) or tracker axis direction
cw = 2 # m . 1-up portrait
pitch=5.7
hub_height=1.5 # m
norm_hub_height = 1.5/cw
norm_pitch = 5.7/cw
rowType = "interior"        
sensorsy = 12                # sampling areas in the module. edges would be 1 and 12.
PVfrontSurface = "glass"    # options: glass or ARglass
PVbackSurface = "glass"     # options: glass or ARglass

# Tracking instructions
tracking=True
backtrack=True
limit_angle = 52

deltastyle = 'exact'  # NSRDB downloads data at center hour 11:30, 12:30, etc... 

In [5]:
#TMYtoread=bifacialvf.getEPW(lat=37.5407,lon=-77.4360)
#myTMY3, meta = bifacialvf.readInputTMY(TMYtoread)

path = C:\Users\sayala\Documents\GitHub\InSPIRE\Studies\BARN_Ground-Sensors
Making path: EPWs
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!


In [9]:
weatherfile2 = r'C:\Users\sayala\Documents\GitHub\InSPIRE\Studies\BARN_Ground-Sensors\WeatherFiles\PSM3_15T.csv'

In [20]:
meta = {'loc': 'LOCATION',
 'city': 'Golden',
 'state-prov': 'CO',
 'country': 'USA',
 'data_type': 'measured',
 'latitude': 39.742,
 'longitude': -105.179,
 'TZ': -7.0,
 'altitude': 1829.0}


In [21]:
df = pd.read_csv(weatherfile2, skiprows=2)

In [26]:
df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
df['datetime'] = df['datetime'].dt.tz_localize('Etc/GMT+7')
df

,Year,Month,Day,Hour,Minute,Wspd,Tdry,DHI,DNI,GHI,Albedo,datetime
datetime,,,,,,,,,,,,
2023-08-28 00:00:00,2023,8,28,0,0,0.570000,15.660000,0.000000,0.067881,0.0,0.0,2023-08-28 00:00:00-07:00
2023-08-28 00:15:00,2023,8,28,0,15,0.499933,15.588000,0.000000,0.261778,0.0,0.0,2023-08-28 00:15:00-07:00
2023-08-28 00:30:00,2023,8,28,0,30,0.065467,15.507333,0.000000,0.322695,0.0,0.0,2023-08-28 00:30:00-07:00
2023-08-28 00:45:00,2023,8,28,0,45,0.482867,15.386667,0.000000,0.072407,0.0,0.0,2023-08-28 00:45:00-07:00
2023-08-28 01:00:00,2023,8,28,1,0,0.124200,15.268000,0.000000,0.142377,0.0,0.0,2023-08-28 01:00:00-07:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-02 23:00:00,2023,11,2,23,0,1.587333,14.622000,0.127996,0.095033,0.0,0.0,2023-11-02 23:00:00-07:00
2023-11-02 23:15:00,2023,11,2,23,15,0.684933,13.648000,0.039862,0.249243,0.0,0.0,2023-11-02 23:15:00-07:00
2023-11-02 23:30:00,2023,11,2,23,30,1.352067,14.110667,0.157131,0.000000,0.0,0.0,2023-11-02 23:30:00-07:00


In [27]:
df.set_index('datetime', inplace=True)

In [29]:
writefiletitle = os.path.join('bifacial_VF_ground.csv')
bifacialvf.simulate(df, meta, writefiletitle=writefiletitle, 
            sazm=sazm, pitch=norm_pitch, hub_height=norm_hub_height, 
         rowType=rowType, sensorsy=sensorsy, 
         PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, 
          tracking=tracking, backtrack=backtrack, agriPV=True,
         limit_angle=limit_angle, deltastyle=deltastyle)



Calculating Sun position with no delta, for exact timestamp in input Weather File
No albedo value set or included in TMY3 file (TMY Column name 'Alb (unitless)' expected) Setting albedo default to 0.2
 
 
********* 
Running Simulation for TMY3: 
Location:   Golden
Lat:  39.742  Long:  -105.179  Tz  -7.0
Parameters: tilt:  0   Sazm:  180     Hub_Height :  0.75   Pitch:  2.85   Row type:  interior   Albedo:  0.2
Saving into bifacial_VF_ground.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.0
Actual distance between rows =  1.85
 
 ***** IMPORTANT --> THIS SIMULATION Has Tracking Activated
Backtracking Option is set to:  True
Saving Ground Irradiance Values for AgriPV Analysis. 


100%|██████████| 6433/6433 [00:28<00:00, 226.32it/s]

Finished
